In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

%matplotlib inline

In [24]:
data = pd.read_csv('../data/test.csv')
Y = data['Y']
data = data.drop(['Y'], axis = 1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, Y, test_size=0.3, random_state = 42)

In [22]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(sampling_strategy = 'minority')
X_over, y_over = ros.fit_resample(X_train, y_train)

In [5]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_under, y_under = rus.fit_resample(X_train, y_train)

In [25]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

tpot = TPOTClassifier(generations = 5, population_size = 20, cv = 3, verbosity=2, scoring = 'f1')

tpot.fit(X_train, y_train)
tpot.score(X_test, y_test)

Generation 1 - Current best internal CV score: 0.15989896254568925
Generation 2 - Current best internal CV score: 0.15989896254568925
Generation 3 - Current best internal CV score: 0.16851194278745238
Generation 4 - Current best internal CV score: 0.16851194278745238
Generation 5 - Current best internal CV score: 0.21009681273500372

Best pipeline: GaussianNB(SelectPercentile(GradientBoostingClassifier(input_matrix, learning_rate=0.001, max_depth=4, max_features=0.35000000000000003, min_samples_leaf=3, min_samples_split=3, n_estimators=100, subsample=0.8), percentile=43))


0.20189274447949523

In [28]:
tpot_config = {
    'sklearn.tree.DecisionTreeClassifier': {
    },

    'sklearn.naive_bayes.MultinomialNB': {
    },

    'sklearn.ensemble.AdaBoostClassifier': {
    }
}
tpot = TPOTClassifier(generations = 5, population_size = 20,cv = 5, verbosity = 2, config_dict = tpot_config, scoring = 'f1')
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

Generation 1 - Current best internal CV score: 0.12243598917541972
Generation 2 - Current best internal CV score: 0.12243598917541972
Generation 3 - Current best internal CV score: 0.12640825030277364
Generation 4 - Current best internal CV score: 0.12640825030277364
Generation 5 - Current best internal CV score: 0.12640825030277364

Best pipeline: DecisionTreeClassifier(MultinomialNB(DecisionTreeClassifier(AdaBoostClassifier(input_matrix))))
0.1342281879194631


In [7]:
tpot = TPOTClassifier(generations = 5, population_size = 50, cv = 5, verbosity=2, scoring = 'f1')

tpot.fit(X_under, y_under)
tpot.score(X_test, y_test)

Generation 1 - Current best internal CV score: 0.6666666666666666
Generation 2 - Current best internal CV score: 0.668643480427507
Generation 3 - Current best internal CV score: 0.668643480427507
Generation 4 - Current best internal CV score: 0.6725138033966981
Generation 5 - Current best internal CV score: 0.6725138033966981

Best pipeline: GaussianNB(StandardScaler(input_matrix))


0.16412213740458012

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV


print("DecisionTreesClassifier")

hyperT = dict(criterion = ["gini","entropy"], max_depth = [None]+[i for i in range(1,100)], max_features = ["auto", "sqrt","log2"]) #,  min_samples_leaf=[i for i in range(1,6)], , min_samples_split = [i for i in range(1,6)]

gridT = GridSearchCV(DecisionTreeClassifier(), hyperT, cv = 3, scoring='f1')
bestT = gridT.fit(X_over, y_over)
y_pred = bestT.predict(X_test)
f1 = f1_score(y_test, y_pred)
print(f"F1: {f1}")
print(gridT.best_params_ )


DecisionTreesClassifier
F1: 0.17355371900826447
{'criterion': 'entropy', 'max_depth': 34, 'max_features': 'auto'}


In [15]:
from sklearn.ensemble import RandomForestClassifier
print("RandomForestClassifier")

hyperT = dict(n_estimators =[i for i in range(10,1000,100)], criterion = ["gini", "entropy"], bootstrap = ["True", "False"])
gridT = GridSearchCV(RandomForestClassifier(), hyperT, cv = 3, scoring='f1')
bestT = gridT.fit(X_over, y_over)
y_pred = bestT.predict(X_test)
f1 = f1_score(y_test, y_pred)
print(f"F1: {f1}")
print(gridT.best_params_ )

RandomForestClassifier
F1: 0.1324503311258278
{'bootstrap': 'False', 'criterion': 'gini', 'n_estimators': 110}


In [18]:
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier(algorithm="auto", n_neighbors=8,p=1, weights="distance")
classifier.fit(X_over, y_over)
y_pred = classifier.predict(X_test)
f1_test = f1_score(y_test, y_pred)
print(f"F1 for non std data: {f1_test}")

F1 for non std data: 0.17659137577002057


In [21]:
from sklearn.tree import DecisionTreeClassifier
print("RandomForestClassifier")

hyperT = dict(activation = ["identity", "logistic", "tanh", "relu"], solver = ["lbfgs", "sgd", "adam"], alpha = [0.0001, 0.001,0.01,0.1], learning_rate = ["constant", "invscaling", "adaptive"])
gridT = GridSearchCV(RandomForestClassifier(), hyperT, cv = 3, scoring='f1')
bestT = gridT.fit(X_over, y_over)
y_pred = bestT.predict(X_test)

f1 = f1_score(y_test, y_pred)

print(f"F1 for non std data: {f1_test}")

F1 for non std data: 0.1395348837209302


In [34]:
from sklearn.neural_network import MLPClassifier

classifier = MLPClassifier(activation = 'lbfgs', solver = 'adam', alpha = 0.001, learning_rate = "adaptive")
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
f1_test = f1_score(y_test, y_pred)
print(f"F1 for non std data: {f1_test}")


ValueError: The activation 'lbfgs' is not supported. Supported activations are ['identity', 'logistic', 'relu', 'softmax', 'tanh'].

In [31]:
hyperT = dict(activation = ["identity", "logistic", "tanh", "relu"], solver = ["lbfgs", "sgd", "adam"], alpha = [0.0001, 0.001,0.01,0.1], learning_rate = ["constant", "invscaling", "adaptive"])
gridT = GridSearchCV(MLPClassifier(), hyperT, cv = 3, scoring='f1')
bestT = gridT.fit(X_over, y_over)
y_pred = bestT.predict(X_test)

f1 = f1_score(y_test, y_pred)

print(f"F1 for non std data: {f1_test}")

/home/joao/.local/lib/python3.7/site-packages/sklearn/utils/extmath.py:151: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/home/joao/.local/lib/python3.7/site-packages/sklearn/utils/extmath.py:151: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_stochastic_optimizers.py:174: RuntimeWarning: invalid value encountered in subtract
  for velocity, grad in zip(self.velocities, grads)]
/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/joao/.local/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:768: RuntimeWarning: invalid value encountered in greater
  y = np.array(y > threshold, dtype=np.int)
/home/joao/.local/lib/python3.7/site-packages/sklearn/utils/extmath.py:151: R

/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/joao/.local/lib/python3.7/site-packages/sklearn/utils/extmath.py:151: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/home/joao/.local/lib/python3.7/site-packages/sklearn/utils/extmath.py:151: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_stochastic_optimizers.py:174: RuntimeWarning: invalid value encountered in subtract
  for velocity, grad in zip(self.velocities, grads)]


/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/joao/.local/lib/python3.7/site-packages/sklearn/utils/extmath.py:151: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/home/joao/.local/lib/python3.7/site-packages/sklearn/utils/extmath.py:151: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_stochastic_optimizers.py:174: RuntimeWarning: invalid value encountered in subtract
  for velocity, grad in zip(self.velocities, grads)]


/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/joao/.local/lib/python3.7/site-packages/sklearn/utils/extmath.py:151: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/home/joao/.local/lib/python3.7/site-packages/sklearn/utils/extmath.py:151: RuntimeWarning: 

/home/joao/.local/lib/python3.7/site-packages/sklearn/utils/extmath.py:151: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_stochastic_optimizers.py:174: RuntimeWarning: invalid value encountered in subtract
  for velocity, grad in zip(self.velocities, grads)]
/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/joao/.local/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:768: RuntimeWarning: invalid value encountered in greater
  y = np.array(y > threshold, dtype=np.int)
/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged y

/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL

/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL

/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL

/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/joao/.local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.ma

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 59 is different from 36)